In [ ]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import scipy.stats as st
import seaborn as sns

In [ ]:
filename = "resources/employers_clean.csv"
employers_df=pd.read_csv(filename)

In [ ]:
employers_df

In [ ]:
employers_reduced = employers_df[employers_df["TICKER"] != 'none'].reset_index(drop=True)

In [ ]:
employers_reduced.head()

In [ ]:
employers_reduced['1 year stock performance']=""
employers_reduced['return on equity']=""
employers_reduced['Analyst Recommendation Mean']=""
employers_reduced['Operating Income']=""


employers_reduced

In [ ]:
print('Finding stock data..')
print("-"*20)
for index,row in employers_reduced.iterrows():
    company=employers_reduced.loc[index,'NAME']
    ticker=employers_reduced.loc[index,'TICKER']
    stock_data=yf.Ticker(ticker)
    try:
        employers_reduced.loc[index, "1 year stock performance"]=round(stock_data.info["52WeekChange"]*100,2)
        employers_reduced.loc[index, "return on equity"]=round(stock_data.info["returnOnEquity"]*100,2)
        employers_reduced.loc[index, "Analyst Recommendation Mean"]=stock_data.info['recommendationMean']
        employers_reduced.loc[index, 'Operating Income']=stock_data.info['ebitda']
    except:
        employers_reduced.loc[index, "1 year stock performance"]="No data found"
        employers_reduced.loc[index, "return on equity"]="No data found"
        employers_reduced.loc[index, "Analyst Recommendation Mean"]="No data found"
        employers_reduced.loc[index, "Operating Income"]="No data found"
        
    print(f"{company} ({ticker}): {employers_reduced.loc[index, '1 year stock performance']}% 1yr stock performance; {employers_reduced.loc[index, 'return on equity']}% RoE; Mean Analyst Rec: {employers_reduced.loc[index, 'Analyst Recommendation Mean']}; EBITDA: {employers_reduced.loc[index, 'Operating Income']}")
    print("-"*20)

employers_reduced

In [ ]:
employers_reduced["Operating Income"].astype(str)
employers_filter=employers_reduced.dropna(axis=0,how='any')
employers_filter=employers_filter.loc[employers_filter['1 year stock performance']!= "No data found"]
employers_filter["Operating Income"].astype(int)
employers_filter["EMPLOYEES"]=employers_filter["EMPLOYEES"].str.replace(',', '').astype(int)


employers_filter.head()



In [ ]:
employers_filter["Operating income per employee"]=employers_filter["Operating Income"]/employers_filter["EMPLOYEES"]
# pd.options.display.float_format = '{:,.2f}'.format
employers_filter["Operating income per employee"]=employers_filter["Operating income per employee"].astype(int)
employers_filter.head()

In [ ]:
stock = employers_filter['1 year stock performance'].astype(float)
stock_z=st.zscore(stock)

roe= employers_filter['return on equity'].astype(float)
roe_z=st.zscore(roe)

op_emp=employers_filter['Operating income per employee'].astype(float)
op_emp_z=st.zscore(op_emp)

analyst=employers_filter['Analyst Recommendation Mean'].astype(float)
analyst_z=st.zscore(analyst)

In [ ]:
employers_filter.head()

In [ ]:
rank=employers_filter["RANK"]
glassdoor=employers_filter["Glassdoor Rating"]
rec=employers_filter["Analyst Recommendation Mean"]
stock=employers_filter["1 year stock performance"]


In [ ]:
fig,axes=plt.subplots(4,2, figsize=(18,14))
fig.suptitle('100 Best Employers: initial financial analysis', size=18, weight='bold')

ax1=sns.scatterplot(ax=axes[0,0], data=employers_filter, x='RANK',y='1 year stock performance',color='red',edgecolor="black")
axes[0,0].invert_xaxis()
axes[0,0].set_ylabel('1 year stock \nperformance (%)',fontsize = 16, weight='bold')
axes[0,0].set_xlabel("'World's Best Employers' ranking",fontsize = 16, weight='bold')
axes[0,0].xaxis.set_label_position('top') 

sns.scatterplot(ax=axes[0,1], data=employers_filter, x='Glassdoor Rating',y='1 year stock performance',color='red',edgecolor="black")

axes[0,1].set_ylabel(' ',fontsize = 16)
axes[0,1].set_xlabel("Glassdoor Overall Rating",fontsize = 16, weight='bold')
axes[0,1].xaxis.set_label_position('top') 

sns.scatterplot(ax=axes[1,0], data=employers_filter, x='RANK',y='return on equity',color='green',edgecolor="black")
axes[1,0].invert_xaxis()
axes[1,0].set_ylabel('Return on equity \n(%)',fontsize = 16, weight='bold')
axes[1,0].set_xlabel(" ",fontsize = 16)


sns.scatterplot(ax=axes[1,1], data=employers_filter, x='Glassdoor Rating',y='return on equity',color='green',edgecolor="black")
axes[1,1].set_ylabel(' ',fontsize = 16)
axes[1,1].set_xlabel(" ",fontsize = 16)


sns.scatterplot(ax=axes[2,0], data=employers_filter, x='RANK',y='Analyst Recommendation Mean',color='gold',edgecolor="black")
axes[2,0].invert_xaxis()
axes[2,0].invert_yaxis()
axes[2,0].set_ylabel('Mean Analyst \n Recommendations',fontsize = 16, weight='bold')
axes[2,0].set_xlabel(" ",fontsize = 16)


sns.scatterplot(ax=axes[2,1], data=employers_filter, x='Glassdoor Rating',y='Analyst Recommendation Mean',color='gold',edgecolor="black")
axes[2,1].set_ylabel(' ',fontsize = 16)
axes[2,1].invert_yaxis()
axes[2,1].set_xlabel(" ",fontsize = 16)

sns.scatterplot(ax=axes[3,0], data=employers_filter, x='RANK',y='Operating income per employee',color='blue',edgecolor="black")
axes[3,0].invert_xaxis()
axes[3,0].set_ylabel('Op. income per \n employee ($)',fontsize = 16, weight='bold')
axes[3,0].set_xlabel(" ",fontsize = 16)


sns.scatterplot(ax=axes[3,1], data=employers_filter, x='Glassdoor Rating',y='Operating income per employee',color='blue',edgecolor="black")
axes[3,1].set_ylabel(" ",fontsize = 16)
axes[3,1].set_xlabel(" ",fontsize = 16)


plt.show()
plt.savefig('Images/Yfinance_scatter.png')


In [ ]:
fig=plt.figure()
ax=fig.add_axes([0,0,1,1])
ax1=ax.scatter(rank,stock_z,color='Red',label='52 week stock performance',edgecolor="black")
ax2=ax.scatter(rank,roe_z,color='green',label='Return on equity')
ax3=ax.scatter(rank,analyst_z,color='gold',label="Mean analysts recommendation",edgecolor="black")
ax4=ax.scatter(rank,op_emp_z,color='blue',label="Operating income per employee")
ax.legend(handles=[ax1,ax2,ax3,ax4],prop={'size':10})
ax.set_xlabel("Rankings", size=14)
ax.set_ylabel("z-scores", size=14)
ax.set_title("Financial performance of top 100 (standardised)",size=16)
ax.invert_xaxis()

plt.show()
plt.savefig('Images/fin_perf_std.png')